In [ ]:
import requests
import os
import sys
import pathlib
import tensorflow as tf
import wave
import base64
tf.config.run_functions_eagerly(True)

In [ ]:
# DATASET CLASS ----------------------------------------------------------------
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id
    
    def read_audio(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        fp = wave.open(file_path)
        nchan = fp.getnchannels()
        N = fp.getnframes()
        audio_bytes = fp.readframes(N * nchan)
        audio_b64bytes = base64.b64encode(audio_bytes.numpy())
        audio_string = audio_b64bytes.decode()
        
        return audio_string, label_id
    

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label, audio_bytes = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label= self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label
    
    def preprocess_big(self, file_path):
        audio_string, label = self.read_audio(file_path)

        return audio_string, label
    
    

    def make_dataset(self, files):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess_with_mfcc, num_parallel_calls=4)
        ds = ds.cache()
        
        ds_raw = tf.data.Dataset.from_tensor_slices(files)
        ds_raw = ds_raw.map(self.preprocess_big, num_parallel_calls=4)
        ds_raw = ds_raw.cache()
       
        return ds, ds_raw



In [ ]:
#creazione dataset per inferenza e per trasmissione

data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

#lista di labels
labels=[]
for el in os.listdir("./data/mini_speech_commands"):
  if el!="README.md":
    labels.append(el)

#lista di test
test_list=[]
with open(os.path.join(sys.path[0], "kws_test_split.txt"), "r") as file:
    for line in file:
        test_list.append('.'+line[1:-1])
        
cnt = 0
for i in test_list:
    #print(i)
    cnt += 1
    if cnt == 5:
        break
        
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,'num_coefficients': 10}

generator = SignalGenerator(labels, 16000, **MFCC_OPTIONS)
test_ds, test_ds_raw = generator.make_dataset(test_list)
    

In [ ]:
for test_sample in test_ds_raw.take(1):
    print(test_sample[0])

In [ ]:
## create the interpreter to run the inference

interpreter = tf.lite.Interpreter(model_path="./model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]




In [ ]:
audio_binary = tf.io.read_file(test_list[0])

In [ ]:
print(audio_binary.numpy())

In [ ]:
#copia da hw2

for test_sample, label in  zip(batch, labels):
            test_sample = np.expand_dims(test_sample, axis=0).astype(np.float32)
            interpreter.set_tensor(input_index, test_sample)
            interpreter.invoke()
            output = interpreter.get_tensor(output_index)
            pred = np.argmax(output)
            if pred == label:
              running_corrects += 1

In [49]:
fp = wave.open(test_list[0])
nchan = fp.getnchannels()
N = fp.getnframes()
audio_bytes = fp.readframes(N * nchan)


audio_binary = tf.io.read_file(test_list[0])
audio, _ = tf.audio.decode_wav(audio_binary)
audio = tf.squeeze(audio, axis=1)
audio = tf.expand_dims(audio, axis=1)


print(audio)


tf.Tensor(
[[0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 ...
 [9.1552734e-05]
 [3.0517578e-05]
 [6.1035156e-05]], shape=(16000, 1), dtype=float32)
